#Decision Tree

each item has two continuous features
class label is {0, 1}



In [79]:
import numpy as np
import pandas as pd
import math

df = pd.read_csv('./data/D1.txt', sep='\s+', names=['x1', 'x2', 'y'])
df.head()

,x1,x2,y
0,0.264185,0.178456,0
1,0.409499,0.213456,1
2,0.926224,0.540329,1
3,0.573685,0.282145,1
4,0.953159,0.608121,1


In [80]:
def determine_candidate_numeric_splits(D: pd.DataFrame, X: str) -> list((str, float)):
    C = []
    instances = D.sort_values(by=X)
    instances = instances.reset_index(drop=True)
    for i, row in instances.iterrows():
        if i < len(instances) - 1:
            y_i = row['y']
            y_next = instances.at[i + 1, 'y']
            if y_i != y_next:
                x_i = row[X]
                C.append((X, x_i))
    return C

In [81]:
def determine_candidate_splits(D: pd.DataFrame) -> list((str, float)):
    C = []
    for feature in D.columns[:-1]:
        C.extend(determine_candidate_numeric_splits(D, feature))
    return C

In [82]:
# GainRatio(D, Split) = (H(Y) - H(Y | Split)) / H(Split)

def entropy_Y(D: pd.DataFrame, Y: str) -> float:
    # H(Y) = -Sum(P(y)*log2(P(y)) for each y in Y)
    H = 0
    y = D[Y].unique()
    for y_i in y:
        Py_i = sum(D[Y] == y_i) / len(D[Y])
        H += Py_i if Py_i == 0 else Py_i * math.log2(Py_i)
    H *= -1
    return H

def entropy_S(D: pd.DataFrame, S: (str, float)) -> float:
    H = 0
    X, c = S
    Pgreater = sum(D[X] >= c) / len(D[X])
    H += Pgreater if Pgreater == 0 else Pgreater * math.log2(Pgreater)
    Plesser = sum(D[X] < c) / len(D[X])
    H += Plesser if Plesser == 0 else Plesser * math.log2(Plesser)
    H *= -1
    return H

def conditional_entropy(D: pd.DataFrame, Y: str, S: (str, float)) -> float:
    # H(Y | Split) = (P(X >= c) * H(Y | X >= c)) + (P(X < c) * H(Y | X < c))
    H = 0
    X, c = S
    HYgreater = entropy_Y(D[D[X] >= c], Y)
    Pgreater = sum(D[X] >= c) / len(D[X])
    HYlesser = entropy_Y(D[D[X] < c], Y)
    Plesser = sum(D[X] < c) / len(D[X])
    H += Pgreater * HYgreater
    H += Plesser * HYlesser
    return H

#print(entropy_Y(df, 'y'))
#print(entropy_S(df, ('x1', .5)))
#print(conditional_entropy(df, 'y', ('x2', .2)))

In [83]:
from dataclasses import dataclass
from typing import List

@dataclass
class Node:
    feature: str = None
    threshold: float = None
    children: List['Node'] = None
    class_label: float = None

In [84]:
def make_subtree(D: pd.DataFrame):
    stop = False
    best_split = None
    candidate_splits = determine_candidate_splits(D)

    # if node is empty or all splits have 0 info or any split has 0 entropy
    if len(D) == 0 or len(candidate_splits) == 0:
        stop = True
    else:
        gainRatios = []
        for split in candidate_splits:
            split_entropy = entropy_S(D, split)
            if split_entropy == 0:
                stop = True
                break
            else:
                # GainRatio(D, Split) = (H(Y) - H(Y | Split)) / H(Split)
                gainRatio = (entropy_Y(D, 'y') - conditional_entropy(D, 'y', split)) / entropy_S(D, split)
                gainRatios.append(gainRatio)
        gainRatios = pd.Series(gainRatios)
        if gainRatios.max() == 0:
            stop = True
        else:
            best_split = candidate_splits[gainRatios.idxmax()]
    if stop:
        # stopping criteria met
        # make leaf node with class label
        majority_class = D['y'].mode()
        if len(majority_class) > 1:
            majority_class = '1'
        else:
            majority_class = majority_class[0]
        new_node = Node(class_label=majority_class)
        return new_node
    else:
        # make internal node and make children subtrees
        feature, threshold = best_split
        left_data = D[D[feature] < threshold]
        right_data = D[D[feature] >= threshold]
        left_child = make_subtree(left_data)
        right_child = make_subtree(right_data)
        new_node = Node(feature, threshold, [left_child, right_child])
        return new_node

In [86]:
dtree = make_subtree(df)